In [2]:
import requests 
import lxml.html as lh
import numpy as np 
import pandas as pd 
import bs4 as bs
import urllib.request
## Source URL
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print ("Imported Libraries")

Imported Libraries


In [3]:
# -----------------------------------------------------
# Using BS4 as suggested in Assignment.
# get_table_bs4 <tableClassName> <expected numberOfColumns>
# -----------------------------------------------------
def get_table_bs4(column,cols):
    reqpage  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(reqpage,'lxml')
    table = soup.find("table",class_=column)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Temp DF to store data
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

In [4]:
# -----------------------------------------------------
# Parsing using xpath.
# -----------------------------------------------------
def get_table_lxml(XPATH,cols):
    reqpage = requests.get(url)
    doc = lh.fromstring(reqpage.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns=headers)
        return raw_df

In [5]:

#Test in beautifulSoup
orgTorontoPC = get_table_bs4("wikitable",3)
print("# Toronto Postal code stored in df")
print(orgTorontoPC.info(verbose=True))

# Toronto Postal codes stored in data
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB
None


In [7]:

# -----------------------------------------------------
# Only process the cells that have an assigned borough. 
# Ignore cells with a borough that is Not assigned.
# -----------------------------------------------------
TorontoPostalCodes=orgTorontoPC[~orgTorontoPC['Borough'].isin(['Not assigned'])]

# Sort and Reset index.
TorontoPostalCodes=TorontoPostalCodes.sort_values(by=['Postal Code','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

In [13]:
# -----------------------------------------------------
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# For example the value of the Borough is North York and the Neighborhood columns will be North York.
# -----------------------------------------------------
TorontoPostalCodes.loc[TorontoPostalCodes['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = TorontoPostalCodes['Borough']
check_unassigned_post_state_sample = TorontoPostalCodes.loc[TorontoPostalCodes['Borough'] == 'North York']

In [14]:
TorontoPostalCodes = TorontoPostalCodes.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
#Exporting TorontoPC to df for other tasks.
TorontoPostalCodes.to_csv('Toronto.TASK_1_df.csv',index=False)
TorontoPostalCodes

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [15]:

TorontoPostalCodes.shape

(103, 3)